In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import os

In [4]:
load_dotenv(override=True)

True

In [6]:
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key and openai_api_key[:8] == "sk-proj-":
    print("OPENAI_API_KEY is present")
else:
    print("OPENAI_API_KEY is not present")

OPENAI_API_KEY is present


In [16]:
openai = OpenAI()

In [63]:
def get_character_profile_information(background, notable_works, occupation, first_appearance, era):
    return {
        "background": background,
        "notable_works": notable_works,
        "occupation": occupation,
        "first_appearance": first_appearance,
        "era": era
    }

In [88]:
get_character_profile_information_json = {
    "name": "get_character_profile_information",
    "description": "Get the character's information like the background, notable works, occupation, first appearance, and era",
    "parameters": {
        "type": "object",
        "properties": {
            "background": {
                "type": "string",
                "description": "Background or the summary of information the character, everything should be factual, and not exceeding 1000 characters."
            },
            "notable_works": {
                "type": "string",
                "description": "A concise summary of the character’s most notable achievements, contributions, or works that made them significant in history or fiction.",
                "example": "Theory of Special Relativity, Theory of General Relativity, Photoelectric Effect, Mass-Energy Equivalence (E=mc²)"
            },
            "occupation": {
                "type": "string",
                "description": "The primary profession, role, or title of the character.",
                "example": "Theoretical Physicist"
            },
            "first_appearance": {
                "type": "string",
                "description": "The date, year, or title of the work/event where the character first appeared or was introduced. This can be a publication, film, historical record, or other medium.",
                "example": "Action Comics #1 (1938)"
            },
            "era": {
                "type": "string",
                "description": "The historical or fictional time period in which the character lived or was most active, including the range of years. Strictly follow the format in the example.",
                "example": "19th Century (1879–1955)"
            }
        },
        "required": ["background", "notable_works", "occupation", "first_appearance", "era"],
        "additionalProperties": False
    }
}

In [89]:
tools = [{"type": "function", "function": get_character_profile_information_json}]

In [90]:
model = "gpt-4o-mini"

In [91]:
character_name = "Albert Einstein"

In [92]:
messages = [
    {"role": "system", "content": "You are a narrator who knows a lot about people and providing character's information as if you really know them."},
    {"role": "user", "content": f"Give me a factual background information of {character_name}. If you don't know this character, just provide N/A and don't mess things up."}
]

In [93]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools
)

message = response.choices[0].message

print(message)


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_1OmL36IxAvnOtqZb44H68tfR', function=Function(arguments='{"background":"Albert Einstein was a theoretical physicist born on March 14, 1879, in Ulm, Germany, and died on April 18, 1955, in Princeton, New Jersey, USA. Renowned for developing the theory of relativity, his work revolutionized the understanding of time, space, and energy. His famous equation E=mc² expressed the equivalence of mass and energy. Einstein\'s early education was marked by a struggle against the rigidities of the German educational system. He later moved to Switzerland, where he obtained citizenship and published several groundbreaking papers in 1905, including those on the photoelectric effect and Brownian motion, which later earned him the Nobel Prize in Physics in 1921. Einstein was also a prominent advocate for civil rights and a passiona

In [94]:
def handle_tool_calls(tool_calls):
    results = {}
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        tool = globals().get(tool_name) if tool_name else { "error": "tool name is not found" }
        result = tool(**arguments)
        results = result
    return results

In [95]:
handle_tool_calls(message.tool_calls)

{'background': "Albert Einstein was a theoretical physicist born on March 14, 1879, in Ulm, Germany, and died on April 18, 1955, in Princeton, New Jersey, USA. Renowned for developing the theory of relativity, his work revolutionized the understanding of time, space, and energy. His famous equation E=mc² expressed the equivalence of mass and energy. Einstein's early education was marked by a struggle against the rigidities of the German educational system. He later moved to Switzerland, where he obtained citizenship and published several groundbreaking papers in 1905, including those on the photoelectric effect and Brownian motion, which later earned him the Nobel Prize in Physics in 1921. Einstein was also a prominent advocate for civil rights and a passionate opponent of nationalism.",
 'notable_works': 'Theory of Relativity, Photoelectric Effect (Nobel Prize), Brownian Motion, Mass-energy equivalence (E=mc²)',
 'occupation': 'Theoretical Physicist',
 'first_appearance': '1905 (in sc